In [1]:
# setup
from pyswmm import Simulation
import os
import dask
import time
from pyswmm.lib import DLL_SELECTION

In [2]:
# swmm library location variable initiation
dll_path = DLL_SELECTION() # get dll path
dll_dir = os.path.dirname(dll_path) # get dll directory
# check to see if the copy exists already
dll_copy_path = dll_path[:-4]+'-copy'+dll_path[-4:] # dll-copy file path

if os.path.exists(dll_copy_path):
    print("Found both copies of the swmm5 dynamic link library file. Carry on.")
else:
    print("ACTION REQUIRED!", 
          "Navigate to <" + dll_dir + ">.", 
          "Copy and paste the file <" + os.path.basename(dll_path) + "> on location.",
          "Make sure there are two copies of <" + os.path.basename(dll_path) + "> (an original and a copy) in the folder.",
          "Rename the duplicate file to '" + os.path.basename(dll_copy_path) + "'.",
          "Then come back, restart the jupyter kernel and try again.", sep="\n")

Found both copies of the swmm5 dynamic link library file. Carry on.


In [3]:
# TWO known, accessible COPIES of the swmm5 dynamic link library files EXIST, so we can only parallelize PAIRS of simmulations.
nsims = 2

In [4]:
# save absolute path of input file location
# this string is how we will tell pyswmm where to locate the files it needs to run its simulations
input_path = os.path.abspath("../probabilistic_python/input/swmm/input_")
print(input_path)

C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_


In [5]:
'''
Makes arrangements for executing the following task:
    Run specified probabilistic simulations in SWMM from the specified .inp file. Save output in equivalently named .out file.
Inputs:
    i <integer> -indicates input folder id number (for now, will only ever be 1 or 2).-
    lib_path <string> -indicates the file path of the swmm5 dynamic link library to be used for running this simmulation.-
Immediate Output:
    <dask.delayed.Delayed> -arrangements for executing the task when commanded at a future time (by the 'compute()' function)
Computable Output:
    <string> -message indicating that the simulation has been completed on the .inp file in the 'i' input folder.-
'''
# this decorator is the first step in using dask to parallelize swmm simulations
@dask.delayed
def delay_job(i, lib_path=None):
    # specify the directory with the file pyswmm needs by attaching the folder id to the rest of the folder's absolute path
    sim_folder = input_path + str(i)
    # specify the actual file pyswmm needs
    sim_file = os.path.join(sim_folder, r'JS_NPlesantCreek.inp')
    print("Simulation input file found:", sim_file)
    # specify the file that pyswmm will (over)write with output after running the probabilistic simulation
    binary_file = sim_folder + r'\JS_NPlesantCreek.out'
    # delete pre-existing .out, if present, in order to run swmm agreeably
    if os.path.exists(binary_file):
        print("Deleting current copy of <JS_NPlesantCreek.out> so new copy can be created.")
        os.remove(binary_file)
    # load the model {no interaction, write (binary) results to <JS_NPlesantCreek.out>, use the specified dll}
    sim = Simulation(inputfile=sim_file, reportfile=None, outputfile=binary_file, swmm_lib_path=lib_path)
    # simulate the loaded model
    sim.execute()
    # a message to indicate success
    return("file " + str(i) + " simulated!")

In [6]:
# can it do 1?
delayed_job = [delay_job(1, lib_path = dll_path)]
delayed_job

[Delayed('delay_job-90c3d97f-2101-4703-bb85-06a7cb87a68f')]

In [7]:
%%time
dask.delayed(print)(delayed_job).compute()

Simulation input file found: C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1\JS_NPlesantCreek.inp
Deleting current copy of <JS_NPlesantCreek.out> so new copy can be created.

... SWMM Version 5.3.0.dev0
... Run Complete['file 1 simulated!']
Wall time: 2min 26s


In [9]:
# can it do multiple?
delayed_jobs = [delay_job(1, lib_path = dll_path),
                delay_job(2, lib_path = dll_copy_path)]
delayed_jobs

[Delayed('delay_job-ffc7013e-1bc9-450d-8712-b6ab656e752a'),
 Delayed('delay_job-43670e68-8c97-41cb-9d68-48c17761dbb1')]

In [10]:
%%time
dask.delayed(print)(delayed_jobs).compute()

Simulation input file found: C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_2\JS_NPlesantCreek.inp
Simulation input file found:Deleting current copy of <JS_NPlesantCreek.out> so new copy can be created.
 C:\Users\Julia Stelman\Desktop\Watershed\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_1\JS_NPlesantCreek.inp
Deleting current copy of <JS_NPlesantCreek.out> so new copy can be created.

... SWMM Version 5.3.0.dev0
... SWMM Version 5.3.0.dev0
... Run Complete
... Run Complete['file 1 simulated!', 'file 2 simulated!']
Wall time: 6min 47s


PySWMM can handle simultaneously running two parallel processes, as long as each one uses its own direct link library. 